In [1]:
!pip uninstall opencv-python-headless==4.5.3.56 --yes


In [2]:
# WandB – Install the W&B library
!pip install wandb -q
!pip install opencv-python==4.1.2.30
!pip install imageio
!pip install onnxruntime
!pip install keras2onnx
!pip install tf2onnx
!pip install pyyaml h5py


Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement opencv-python==4.1.2.30 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.14.51, 3.4.14.53, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 3.4.17.63, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.2.52, 4.5.2.54, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64)
ERROR: No matching distribution found for opencv-python==4.1.2.30
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical

import wandb
from wandb.keras import WandbCallback

import tf2onnx
import onnx

import imageio, glob, cv2
import matplotlib.pyplot as plt
%matplotlib inline

2022-04-03 16:58:14.777703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-03 16:58:14.777758: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
(train_images, train_labels),(test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Reshape the images.
train_images_re = np.expand_dims(train_images, axis=3)
test_images_re = np.expand_dims(test_images, axis=3)

classes = np.unique(train_labels)
input_shape = train_images_re[0].shape

print(classes)
print(input_shape)


[0 1 2 3 4 5 6 7 8 9]
(28, 28, 1)


In [6]:
#plt.figure(figsize=(10,10))
#plt.subplots_adjust(wspace=0, hspace=0)
#i = 0
#for item in train_images[:25]:
#    print(item[0].shape)
#    w, h = 28, 28
#    data = np.zeros((h, w, 1), dtype=np.uint8)
#    data[0:28, 0:28] = [item[0], item[1], 0]
    #img = cv2.imread(item)
    #img = cv2.resize(img, (250, 250))
    #plt.axis('off')
    #plt.subplot(5, 5, i+1) #.set_title(l)
    #plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#    i += 1

In [7]:
def createModel(input_shape, num_classes):
    model = Sequential([keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(num_classes)])
    return model

In [8]:
model = createModel(input_shape, len(classes))

2022-04-03 16:58:18.640892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-03 16:58:18.640951: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-03 16:58:18.640979: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (driver): /proc/driver/nvidia/version does not exist
2022-04-03 16:58:18.641304: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [10]:
num_epochs=10
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images_re, train_labels, epochs=num_epochs, validation_split=0.2, verbose = 1)

Epoch 1/10
1500/1500 [==============================] - 47s 31ms/step - loss: 0.5971 - accuracy: 0.8082 - val_loss: 0.3992 - val_accuracy: 0.8539
Epoch 2/10
1500/1500 [==============================] - 45s 30ms/step - loss: 0.3625 - accuracy: 0.8694 - val_loss: 0.3565 - val_accuracy: 0.8676
Epoch 3/10
1500/1500 [==============================] - 45s 30ms/step - loss: 0.3184 - accuracy: 0.8834 - val_loss: 0.3338 - val_accuracy: 0.8818
Epoch 4/10
1500/1500 [==============================] - 45s 30ms/step - loss: 0.2929 - accuracy: 0.8920 - val_loss: 0.3163 - val_accuracy: 0.8863
Epoch 5/10
1500/1500 [==============================] - 45s 30ms/step - loss: 0.2722 - accuracy: 0.8992 - val_loss: 0.3135 - val_accuracy: 0.8884
Epoch 6/10
1500/1500 [==============================] - 45s 30ms/step - loss: 0.2562 - accuracy: 0.9038 - val_loss: 0.3071 - val_accuracy: 0.8900
Epoch 7/10
1500/1500 [==============================] - 45s 30ms/step - loss: 0.2420 - accuracy: 0.9092 - val_loss: 0.3186 -

In [11]:
test_loss, test_acc = model.evaluate(test_images_re,  test_labels, verbose=2)

313/313 - 3s - loss: 0.3705 - accuracy: 0.8826 - 3s/epoch - 9ms/step


In [17]:
#input_sig = [tf.TensorSpec([3, 3], name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model,
                                           #input_sig,
                                           opset=13)
#onnx.save(onnx_model, "models/onnx_mdl_01.onnx")
onnx.save(onnx_model, 'models/model.onnx')

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2022-04-03 17:13:16.851937: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-03 17:13:16.852154: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-04-03 17:13:16.855055: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.012ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-04-03 17:13:16.917731: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-04-03 17:13:16.917916: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2022-04-03 17:13:16.926244: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1149] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 31 nodes (-10), 40 edges 

In [14]:
model.save('models/model.h5')

In [16]:
model1 = keras.models.load_model('models/model.h5')
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0